# <center>Big Data &ndash; Exercises</center>
## <center>Fall 2024 &ndash; Week 13 &ndash; ETH Zurich</center>
## <center>Cubes</center>

## Introduction

In the first part of this exercise, we will use a spreadsheet application to analyze the sales data of a fictious wholesale supplier (taken from the database system benchmark [TPC-H](http://www.tpc.org/tpch/)). Then, we will use SQL to query the data shape cube. 
A cube is a collection of numeric data organized by arrays of discrete identifiers (Janus and Fouché, 2009).

## 1. The TPC-H Dataset as OLAP Cube

Let us get familiar with the dataset.
It consists of **orders**, each of which is made by a **customer**, and consists of **lineitems**.
Think of an order as a shopping cart with several items in it.
The items of an order are **parts** that may be provided by different **suppliers**.
Suppliers and customers come from different **nations**, which are grouped into **regions** of the world.
The following figure illustrates the schema of the TPC-H dataset.
<br>

Display

![Schema of the TPC-H dataset](https://www.polybox.ethz.ch/index.php/s/ggS3c4d7LFtxEjH/download)


### Task 1

Bruh

1. Which table(s) of the TPC-H schema is/are the fact table(s)?
1. What is/are the measure(s)?
1. What are the dimensions?
1. What do you call this flavor of OLAP?

### Solutions

WTF are these questions?

1. Depends
2. Depends
3. Each table name?

## 2. Analyzing TPC-H with a Pivot Table

Open local [Exercise13_OLAP_Cubes](./Exercise13_OLAP_Cubes.xls) with your favorite spreadsheet application.
The file contains a universal table (a fully denormalized table) of a small TPC-H dataset.
The schema has been modified slightly to make analysis in a spreadsheet application easier:
The two precomputed measures revenue and cost
as well as the hierarchy of time dimensions in the attribute *orderdate* have been added in the materialized form
and some other attributes have been removed.

You may need to look up how to use pivot tables in your spreadsheet application.

1. Microsoft Excel: [PivotTable](https://support.office.com/en-us/article/Create-a-PivotTable-to-analyze-worksheet-data-a9a84538-bfe9-40a9-a8e9-f99134456576)
1. Google Sheets  : [pivot tables](https://support.google.com/docs/answer/1272900?co=GENIE.Platform%3DDesktop&hl=en)
1. Open Office    : [DataPilot](https://openoffice.blogs.com/openoffice/2006/11/data_pilots_in_.html)

### Task 1: Discussion

Discuss the terms "slice and dice", "drill down", "roll up", and "pivoting".

### Solution: Discussion

...

### Task 2: Create the following pivot tables:

1. Show how much revenue suppliers from different regions (as columns) produced in every year (as rows).
1. Show how much revenue suppliers from nations of Africa (as columns) produced in every year (as rows).
1. Show how much revenue suppliers from nations of Africa produced in every quarter of every year.
1. Show how much revenue suppliers from nations of Africa produced in every week of every month of Q1 in 1996.
1. Show how much revenue suppliers from nations of Africa produced in every year with "urgent" orders.
1. Show the average order quantity for parts from suppliers from nations of Africa per year.
1. Show how much revenue suppliers from nations of Africa (as rows) produced in every quarter of every year (as columns).

### Solution:
...

## 3. OLAP Cubes and SQL

Write SQL queries for the PivotTables from Question 2.

#### Notes

* Assume that the revenue is calculated as `olquantity * partretailprice * (1-oldiscount)`. You will already find it as a column.
* To get the year or quarter from a date in PostgreSQL, you can use [`DATE_PART ('field', date )  `](https://www.postgresqltutorial.com/postgresql-date_part/). Note that the field is case-insensitive. You can write `DATE_PART('YEAR', date)` or `DATE_PART('year', date)`, which are equivalent. 

#### Database Set-up

Just like any other week, you need to `docker compose up` and please wait for the message `PostgreSQL init process complete; ready for start up` before proceeding!
As before, we set up our connection to the database and enable use of `%sql` and `%%sql`.

In [8]:
server='db'
user='postgres'
password='bigdataclass'
database='tpch-db'

connection_string = f'postgresql://{user}:{password}@{server}:5432/{database}'

In [9]:
%reload_ext sql
%sql $connection_string

Check the tables in TPC-H. They are empty for the moment.

In [10]:
%%sql 
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';

 * postgresql://postgres:***@db:5432/tpch-db
8 rows affected.


KeyError: 'DEFAULT'

Populate the tables in TPC-H with data from .tbl files

In [ ]:
import numpy as np
import os
import pandas
import psycopg2
import sqlalchemy
import sys

DB_PARAMS = {
    "host": server,               
    "port": 5432,              
    "dbname": database,        
    "user": user,         
    "password": password  
}

tables = [ # Order is important because of FKs
        'region',
        'nation',
        'supplier',
        'part',
        'supplypart',
        'customer',
        'orders',
        'orderline'
        ]
engine = sqlalchemy.create_engine(connection_string)
try:
    conn = psycopg2.connect(**DB_PARAMS)
    cursor = conn.cursor()
    print("Connected to the database!")
except Exception as e:
    print(f"Error connecting to the database: {e}")
    raise

query = """
    SELECT column_name
    FROM information_schema.columns
    WHERE table_name = %s
    ORDER BY ordinal_position;
"""

for table in tables:
    # Find column names
    print(table)
    cursor.execute(query, (table,))
    results = cursor.fetchall()
    columns = []
    for i in results:
        columns.append(i[0])
        
    data = pandas.read_csv('./data/{0}.tbl'.format(table), sep='|', header=None, names=columns)
    msg = 'Loading table "{0}": {1}% done\r'
    for idx, chunk in enumerate(np.array_split(data, 100)):
        sys.stdout.write(msg.format(table, idx))
        chunk.to_sql(name=table, if_exists='append', con=engine, index=False, method='multi')
    print(msg.format(table, str(100)))

First however, we define the fact table using a WITH statement (copy this at the beginning of all other queries)

In [ ]:
%%sql
WITH cube AS (
    SELECT olquantity, partretailprice, oldiscount,
           orderdate, nationname, regionname, orderpriority,
           olquantity * partretailprice * (1-oldiscount) AS revenue,
           DATE_PART('year', orderdate) AS orderyear,
           DATE_PART('quarter', orderdate) AS orderquarter,
           DATE_PART('month', orderdate) AS ordermonth,
           DATE_PART('week', orderdate) AS orderweek
    FROM orderline ol
    JOIN orders o      ON ol.orderid = o.orderid
    JOIN supplypart sp ON ol.partId = sp.partId AND ol.supplierId = sp.supplierId
    JOIN part p        ON sp.partId = p.partId
    JOIN supplier s    ON sp.supplierId = s.supplierId
    JOIN nation sn     ON s.nationId = sn.nationId
    JOIN region sr     ON sn.regionId = sr.regionId
)
SELECT * FROM cube LIMIT 10

Note that, for the purpose of this exercise, we dropped some dimensions of the cube because none of the queries uses them. Also, we materialize some hierarchy levels of the `orderdate` dimension in order to make the subsequent queries more readable. This makes them *look* like they were new dimensions -- conceptually, they are not! (They are, well, levels of a hierarchy of the `orderdate` dimension.)

OK, you are good to go. Use the SQL cell below and add more cells as you need.

Note that the numbers you obtain with the SQL queries should not be identical to those in the pivot tables in Task2, because the data we have in the DB have more rows in its fact table.

#### Your Answers

#### 1. Show how much revenue suppliers from different regions (as columns) produced in every year (as rows).

In [ ]:
%%sql
WITH cube AS (
    SELECT olquantity, partretailprice, oldiscount,
           orderdate, nationname, regionname, orderpriority,
           olquantity * partretailprice * (1-oldiscount) AS revenue,
           DATE_PART('year', orderdate) AS orderyear,
           DATE_PART('quarter', orderdate) AS orderquarter,
           DATE_PART('month', orderdate) AS ordermonth,
           DATE_PART('week', orderdate) AS orderweek
    FROM orderline ol
    JOIN orders o      ON ol.orderid = o.orderid
    JOIN supplypart sp ON ol.partId = sp.partId AND ol.supplierId = sp.supplierId
    JOIN part p        ON sp.partId = p.partId
    JOIN supplier s    ON sp.supplierId = s.supplierId
    JOIN nation sn     ON s.nationId = sn.nationId
    JOIN region sr     ON sn.regionId = sr.regionId
)
...

#### 2. Show how much revenue suppliers from nations of Africa produced in every year.

In [ ]:
%%sql
WITH cube AS (
    SELECT olquantity, partretailprice, oldiscount,
           orderdate, nationname, regionname, orderpriority,
           olquantity * partretailprice * (1-oldiscount) AS revenue,
           DATE_PART('year', orderdate) AS orderyear,
           DATE_PART('quarter', orderdate) AS orderquarter,
           DATE_PART('month', orderdate) AS ordermonth,
           DATE_PART('week', orderdate) AS orderweek
    FROM orderline ol
    JOIN orders o      ON ol.orderid = o.orderid
    JOIN supplypart sp ON ol.partId = sp.partId AND ol.supplierId = sp.supplierId
    JOIN part p        ON sp.partId = p.partId
    JOIN supplier s    ON sp.supplierId = s.supplierId
    JOIN nation sn     ON s.nationId = sn.nationId
    JOIN region sr     ON sn.regionId = sr.regionId
)
...

#### 3. Show how much revenue suppliers from nations of Africa produced in every quarter of every year.

In [ ]:
%%sql
WITH cube AS (
    SELECT olquantity, partretailprice, oldiscount,
           orderdate, nationname, regionname, orderpriority,
           olquantity * partretailprice * (1-oldiscount) AS revenue,
           DATE_PART('YEAR', orderdate) AS orderyear,
           DATE_PART('QUARTER', orderdate) AS orderquarter,
           DATE_PART('MONTH', orderdate) AS ordermonth,
           DATE_PART('WEEK', orderdate) AS orderweek
    FROM orderline ol
    JOIN orders o      ON ol.orderid = o.orderid
    JOIN supplypart sp ON ol.partId = sp.partId AND ol.supplierId = sp.supplierId
    JOIN part p        ON sp.partId = p.partId
    JOIN supplier s    ON sp.supplierId = s.supplierId
    JOIN nation sn     ON s.nationId = sn.nationId
    JOIN region sr     ON sn.regionId = sr.regionId
)
...

#### 4. Show how much revenue suppliers from nations of Africa produced in every week of every month of Q1 in 1996.

Note that `orderweek` is from a different hierarchy of the `orderdate` dimension than `orderquarter` and `ordermonth` because a week does not generally belong to only one quarter or month. (However, a month always belongs to exactly one quarter.) This does not change anything in the SQL query below, but is an important conceptual subtlety of cubes.

In [ ]:
%%sql
WITH cube AS (
    SELECT olquantity, partretailprice, oldiscount,
           orderdate, nationname, regionname, orderpriority,
           olquantity * partretailprice * (1-oldiscount) AS revenue,
           DATE_PART('year', orderdate) AS orderyear,
           DATE_PART('quarter', orderdate) AS orderquarter,
           DATE_PART('month', orderdate) AS ordermonth,
           DATE_PART('week', orderdate) AS orderweek
    FROM orderline ol
    JOIN orders o      ON ol.orderid = o.orderid
    JOIN supplypart sp ON ol.partId = sp.partId AND ol.supplierId = sp.supplierId
    JOIN part p        ON sp.partId = p.partId
    JOIN supplier s    ON sp.supplierId = s.supplierId
    JOIN nation sn     ON s.nationId = sn.nationId
    JOIN region sr     ON sn.regionId = sr.regionId
)
...

#### 5. Show how much revenue suppliers from nations of Africa produced in every year with "urgent" orders.

In [ ]:
%%sql
WITH cube AS (
    SELECT olquantity, partretailprice, oldiscount,
           orderdate, nationname, regionname, orderpriority,
           olquantity * partretailprice * (1-oldiscount) AS revenue,
           DATE_PART('YEAR', orderdate) AS orderyear,
           DATE_PART('QUARTER', orderdate) AS orderquarter,
           DATE_PART('MONTH', orderdate) AS ordermonth,
           DATE_PART('WEEK', orderdate) AS orderweek
    FROM orderline ol
    JOIN orders o      ON ol.orderid = o.orderid
    JOIN supplypart sp ON ol.partId = sp.partId AND ol.supplierId = sp.supplierId
    JOIN part p        ON sp.partId = p.partId
    JOIN supplier s    ON sp.supplierId = s.supplierId
    JOIN nation sn     ON s.nationId = sn.nationId
    JOIN region sr     ON sn.regionId = sr.regionId
)
...

#### 6. Show the average order quantity for parts from suppliers from nations in Africa per year.

In [ ]:
%%sql
WITH cube AS (
    SELECT olquantity, partretailprice, oldiscount,
           orderdate, nationname, regionname, orderpriority,
           olquantity * partretailprice * (1-oldiscount) AS revenue,
           DATE_PART('YEAR', orderdate) AS orderyear,
           DATE_PART('QUARTER', orderdate) AS orderquarter,
           DATE_PART('MONTH', orderdate) AS ordermonth,
           DATE_PART('WEEK', orderdate) AS orderweek
    FROM orderline ol
    JOIN orders o      ON ol.orderid = o.orderid
    JOIN supplypart sp ON ol.partId = sp.partId AND ol.supplierId = sp.supplierId
    JOIN part p        ON sp.partId = p.partId
    JOIN supplier s    ON sp.supplierId = s.supplierId
    JOIN nation sn     ON s.nationId = sn.nationId
    JOIN region sr     ON sn.regionId = sr.regionId
)
...

#### 7. Show how much revenue suppliers from nations of Africa (as rows) produced in every quarter of every year (as columns).

Columns and row of a cube are both represented as columns when mapped to relations and SQL. A tool similar to Excel's PivotTable that automatically generates SQL queries would probably just flip the order of the columns.

In [ ]:
%%sql
WITH cube AS (
    SELECT olquantity, partretailprice, oldiscount,
           orderdate, nationname, regionname, orderpriority,
           olquantity * partretailprice * (1-oldiscount) AS revenue,
           DATE_PART('YEAR', orderdate) AS orderyear,
           DATE_PART('QUARTER', orderdate) AS orderquarter,
           DATE_PART('MONTH', orderdate) AS ordermonth,
           DATE_PART('WEEK', orderdate) AS orderweek
    FROM orderline ol
    JOIN orders o      ON ol.orderid = o.orderid
    JOIN supplypart sp ON ol.partId = sp.partId AND ol.supplierId = sp.supplierId
    JOIN part p        ON sp.partId = p.partId
    JOIN supplier s    ON sp.supplierId = s.supplierId
    JOIN nation sn     ON s.nationId = sn.nationId
    JOIN region sr     ON sn.regionId = sr.regionId
)
...